In [1]:
import pandas as pd
import numpy as np

### Columns which I think are not useful or hard to utilize: 

- year: Storm year
- county: Impacted county name
- abrState: State postal abreviation
- name: Storm name
- numInMonth: Storm sequence in storm's month 1 (all 1's)
- numInSeason: Storm sequence in storm's season (all 1's)
- recoverCount: Storm sequence in previous storm's recovery period (all 1's)
- takeupTotal: County take up rate in month 0

### One-hot encoding variables: 

- month: Storm month
- CAT: Storm category over county

In [2]:
raw = pd.read_csv('sampleUnstr.csv')

# Drop columns mentioned above
raw.drop(['year','county','abrState','name','numInMonth', 'numInSeason','recoverCount','takeupTotal'], axis=1, inplace=True)

# One-hot encoding
dummy_month = pd.get_dummies(raw.month)
dummy_CAT = pd.get_dummies(raw.CAT)
data = pd.concat([raw, dummy_month, dummy_CAT], axis=1, sort=False)
data.drop(['month','CAT'], axis=1, inplace=True)

In [3]:
data.columns

Index([           'recovery',            'prevProd',          'Production',
                      'vmax',                'mslp',                'time',
               'ratePoverty',    'houseMedianValue',       'houseOccupied',
                'houseTotal', 'sumBuildingCoverage',         'policyCount',
                           1,                     2,                     3,
                           4,                     5,                     6,
                           7,                     8,                     9,
                          10,                    11,                    12,
                        'EX',                  'H1',                  'H2',
                        'H3',                  'H4',                  'H5',
                        'LO',                  'SS',                  'TD',
                        'TS'],
      dtype='object')

### Models 

2-layer model framework: <br>
  First layer: classification for whether obs is an outlier (set threshold = 12, minority class ratio = 10%) <br>
  Second layer: regression

In [4]:
#Train valid test split
from sklearn.model_selection import train_test_split

def outlier_label(df, i, threshold):
    if df.iloc[i].recovery > threshold:
        return 1
    else:
        return 0

data['greater_2'] = [outlier_label(data, i, 2) for i in range(len(data))]
data['outlier'] = [outlier_label(data, i, 11) for i in range(len(data))]

In [5]:
X, y = data.drop(['recovery'], axis=1), data[['recovery', 'outlier', 'greater_2']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
X_train.columns

Index([           'prevProd',          'Production',                'vmax',
                      'mslp',                'time',         'ratePoverty',
          'houseMedianValue',       'houseOccupied',          'houseTotal',
       'sumBuildingCoverage',         'policyCount',                     1,
                           2,                     3,                     4,
                           5,                     6,                     7,
                           8,                     9,                    10,
                          11,                    12,                  'EX',
                        'H1',                  'H2',                  'H3',
                        'H4',                  'H5',                  'LO',
                        'SS',                  'TD',                  'TS',
                 'greater_2',             'outlier'],
      dtype='object')

### 1st layer (a): classification random forest for recovery = 2

Output model name: **layer1a_rf**

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_curve, auc
from sklearn.metrics import classification_report

X_train_1 = X_train.drop(['outlier', 'greater_2'], axis=1)
y_train_1 = X_train['greater_2']
X_test_1 = X_test.drop(['outlier', 'greater_2'], axis=1)
y_test_1 = X_test['greater_2']

parameters = {'n_estimators':[50,100], 'max_depth':[None, 5, 10]}
layer1a_rf = GridSearchCV(RandomForestClassifier(random_state=1, oob_score=True, class_weight='balanced_subsample'), parameters, scoring='f1')
layer1a_rf.fit(X_train_1, y_train_1)

y_pred_rf = layer1a_rf.predict(X_test_1)
y_pred_proba_rf = layer1a_rf.predict_proba(X_test_1)[::,1]
fpr_rf, tpr_rf, thresholds_rf = roc_curve(y_test_1, y_pred_proba_rf)
auc_rf = auc(fpr_rf, tpr_rf)

print(classification_report(y_test_1, y_pred_rf))
print('F1 score:', f1_score(y_test_1,y_pred_rf))
print('AUC:', auc_rf)
print('Accuracy:', accuracy_score(y_test_1,y_pred_rf))

              precision    recall  f1-score   support

           0       1.00      0.84      0.91        95
           1       0.96      1.00      0.98       395

    accuracy                           0.97       490
   macro avg       0.98      0.92      0.95       490
weighted avg       0.97      0.97      0.97       490

F1 score: 0.9813664596273292
AUC: 0.9987608261159227
Accuracy: 0.9693877551020408


### 1st layer (b): classification random forest for outlier (recovery >= 12)

Output model name: **layer1b_rf**

In [8]:
X_train_1b = X_train[X_train.greater_2 == 1].drop(['outlier', 'greater_2'], axis=1)
y_train_1b = X_train[X_train.greater_2 == 1]['outlier']
X_test_1b = X_test[X_test.greater_2 == 1].drop(['outlier', 'greater_2'], axis=1)
y_test_1b = X_test[X_test.greater_2 == 1]['outlier']

parameters = {'n_estimators':[50,100], 'max_depth':[None, 5, 10]}
layer1b_rf = GridSearchCV(RandomForestClassifier(random_state=1, oob_score=True, class_weight='balanced_subsample'), parameters, scoring='f1')
layer1b_rf.fit(X_train_1b, y_train_1b)

y_pred_rf = layer1b_rf.predict(X_test_1b)
y_pred_proba_rf = layer1b_rf.predict_proba(X_test_1b)[::,1]
fpr_rf, tpr_rf, thresholds_rf = roc_curve(y_test_1b, y_pred_proba_rf)
auc_rf = auc(fpr_rf, tpr_rf)

print(classification_report(y_test_1b, y_pred_rf))
print('F1 score:', f1_score(y_test_1b, y_pred_rf))
print('AUC:', auc_rf)
print('Accuracy:', accuracy_score(y_test_1b,y_pred_rf))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       157
           1       1.00      1.00      1.00       238

    accuracy                           1.00       395
   macro avg       1.00      1.00      1.00       395
weighted avg       1.00      1.00      1.00       395

F1 score: 1.0
AUC: 1.0
Accuracy: 1.0


### Second layer (a): non-outliers (recovery in [3,11])

Decision Tree Regression, Random Forest Regression, Gradient Boosting Regression <br>
All conducted on 5 folds cross-validation

In [9]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

X_train_2a = X_train[X_train['outlier']==0][X_train['greater_2']==1]
y_train_2a = y_train[y_train['outlier']==0][y_train['greater_2']==1]['recovery']
X_train_2a.drop(['outlier', 'greater_2'], axis=1, inplace=True)

# DT
layer2a_dt = DecisionTreeRegressor(max_depth=5, random_state=0)
print('Decision Tree R2:', cross_val_score(layer2a_dt, X_train_2a, y_train_2a, scoring='r2').mean())

# RF
layer2a_rf = RandomForestRegressor(max_depth=5, random_state=0)
print('Random Forest R2:', cross_val_score(layer2a_rf, X_train_2a, y_train_2a, scoring='r2').mean())

# GB
layer2a_gb = GradientBoostingRegressor(random_state=0)
print('Gradient Boosting R2:', cross_val_score(layer2a_gb, X_train_2a, y_train_2a, scoring='r2').mean())

/Users/yuweiwang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/yuweiwang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


Decision Tree R2: 0.5512940608247604
Random Forest R2: 0.6251368332850928
Gradient Boosting R2: 0.806763200980084


### Second layer (b): outliers

In [10]:
X_train_2b = X_train[X_train['outlier']==1][X_train['greater_2']==1]
y_train_2b = y_train[y_train['outlier']==1]['recovery']
X_train_2b.drop(['outlier', 'greater_2'], axis=1, inplace=True)

# DT
layer2b_dt = DecisionTreeRegressor(max_depth=5, random_state=0)
print('Decision Tree R2:', cross_val_score(layer2b_dt, X_train_2b, y_train_2b, scoring='r2').mean())

# RF
layer2b_rf = RandomForestRegressor(max_depth=5, random_state=0)
print('Random Forest R2:', cross_val_score(layer2b_rf, X_train_2b, y_train_2b, scoring='r2').mean())

# GB
layer2b_gb = GradientBoostingRegressor(random_state=0)
print('Gradient Boosting R2:', cross_val_score(layer2b_gb, X_train_2b, y_train_2b, scoring='r2').mean())

/Users/yuweiwang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


Decision Tree R2: 0.9911196355046821
Random Forest R2: 0.9938149799229089
Gradient Boosting R2: 0.9993397764999965


### Test Algorithm

In [11]:
X_test.drop(['outlier', 'greater_2'], axis=1, inplace=True)
y_test.drop(['outlier', 'greater_2'], axis=1, inplace=True)

/Users/yuweiwang/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [14]:
def prediction(X_test, greater2_ind, outlier_ind):
    '''
    The logic is as follows: we first predict all observations with both layer-2 models. 
                             then depends on output of layer-1 classifications, append values accordingly
    For 2nd layer: 2(a) use gradient boosting, 2(b) use random forest
    '''
    y_pred = []
    
    layer2a_gb.fit(X_train_2a, y_train_2a)
    y_pred_a = layer2a_gb.predict(X_test)
    
    layer2b_rf.fit(X_train_2b, y_train_2b)
    y_pred_b = layer2b_rf.predict(X_test)
    
    for i in range(len(X_test)):
        if greater2_ind[i] == 0:
            y_pred.append(2)
        elif outlier_ind[i] == 0:
            y_pred.append(y_pred_a[i])
        elif outlier_ind[i] == 1:
            y_pred.append(y_pred_b[i])
    return y_pred

In [15]:
from sklearn.metrics import mean_squared_error, r2_score

greater2_pred = layer1a_rf.predict(X_test)
outlier_pred = layer1b_rf.predict(X_test)
y_pred = prediction(X_test, greater2_pred, outlier_pred)

print('Test R2 score:', r2_score(y_test,y_pred))
print('Test MSE:', mean_squared_error(y_test, y_pred))

Test R2 score: 0.9927730324038248
Test MSE: 6.463153273812686


#### Comparison with single layer regression

In [16]:
X_train_single = X_train.drop(['outlier','greater_2'], axis=1)
y_train_single = y_train.drop(['outlier','greater_2'], axis=1)

single_rf = RandomForestRegressor(random_state=1, max_depth=5)
single_rf.fit(X_train_single, y_train_single)
y_pred_single = single_rf.predict(X_test)

print('Single Layer R2 score:', r2_score(y_test,y_pred_single))
print('Single Layer MSE:', mean_squared_error(y_test, y_pred_single))

/Users/yuweiwang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


Single Layer R2 score: 0.9293522567472512
Single Layer MSE: 63.18102122571221
